<h1 align=center>Stock Analysis</h1>


### `Points`:

- For 3 months to 1 year, you get a balanced view that works for both short-term traders and AI-based market predictions.

- Longer durations (1+ years) are better suited for long-term investors or deeper historical trend analysis.


### Tools

1. fetch data from yahoo finance for 12 months, cause we want short term trading


In [13]:
# tool: fetch stock price

from typing import Union, Dict
import datetime as dt
import yfinance as yf


def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches historical stock price data and technical indicators for a given ticker."""
    try:
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=48),
            end=dt.datetime.now(),
            interval='1wk'
        )

        return data

    except Exception as e:
        return f"Error fetching price data: {str(e)}"

In [14]:
ticker = "AAPL"  # Example: Apple Inc.

# Fetch historical data (e.g., past 3 months)
data = get_stock_prices(ticker)

# Display the fetched data
data.head()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,


In [ ]:
data["Close"].squeeze()

In [ ]:
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.volume import volume_weighted_average_price

indicators = {}

In [ ]:
data.reset_index(inplace=True)
data['Date'] = data['Date'].astype(str)

In [ ]:
rsi_series = RSIIndicator(data['Close'].squeeze(), window=14).rsi().iloc[-12:]
indicators["RSI"] = dict(
    zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), rsi_series)))

In [ ]:
indicators["RSI"]

In [ ]:
StochasticOscillator(data['High'].squeeze(), data['Low'].squeeze(
), data['Close'].squeeze(), window=14).stoch().iloc[-8:]

In [ ]:
sto_series = StochasticOscillator(data['High'].squeeze(), data['Low'].squeeze(
), data['Close'].squeeze(), window=14).stoch().iloc[-12:]
indicators["Stochastic_Oscillator"] = dict(
    zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), sto_series)))

In [ ]:
indicators["Stochastic_Oscillator"]

In [ ]:
macd = MACD(data['Close'].squeeze())
macd_series = macd.macd().iloc[-12:]
indicators["MACD"] = dict(
    zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), macd_series)))

In [ ]:
indicators["MACD"]

In [ ]:
macd_signal_series = macd.macd_signal().iloc[-12:]
indicators["MACD_Signal"] = dict(
    zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), macd_signal_series)))

In [ ]:
indicators["MACD_Signal"]

In [ ]:
vwap_series = volume_weighted_average_price(
    data['High'].squeeze(), data['Low'].squeeze(), data['Close'].squeeze(), volume=data['Volume'].squeeze()
).iloc[-12:]
indicators["vwap"] = dict(
    zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), vwap_series)))

In [ ]:
indicators["vwap"]

In [1]:
# tool: fetch stock price

from typing import Union, Dict
import datetime as dt
import pandas as pd
import yfinance as yf
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from ta.volume import volume_weighted_average_price
from langchain_core.tools import tool


@tool
def get_stock_prices(ticker: str) -> Union[Dict, str]:
    """Fetches historical stock price data and technical indicators for a given ticker."""
    try:
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(weeks=12),
            end=dt.datetime.now(),
            interval='1wk'
        )

        if data.empty:
            return f"No data found for ticker: {ticker}"

        # reset index so we can access 'Date' as a column
        data.reset_index(inplace=True)
        data['Date'] = data['Date'].astype(str)

        # Technical Indicators - computed on closing prices
        indicators = {}

        # RSI detects overbought/oversold conditions
        # Show last 12 weeks of indicators for a 3-month span
        rsi_series = RSIIndicator(
            data['Close'].squeeze(), window=14).rsi().iloc[-12:]
        indicators["RSI"] = dict(
            zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), rsi_series)))

        # Compares current price to a range of previous prices
        # Another momentum indicator — complements RSI
        sto_series = StochasticOscillator(data['High'].squeeze(), data['Low'].squeeze(
        ), data['Close'].squeeze(), window=14).stoch().iloc[-12:]
        indicators["Stochastic_Oscillator"] = dict(
            zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), sto_series)))

        # MACD is a trend-following indicator (difference of two EMAs)
        # Useful for spotting trend reversals
        macd = MACD(data['Close'].squeeze())
        macd_series = macd.macd().iloc[-12:]
        indicators["MACD"] = dict(
            zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), macd_series)))

        # Signal line is a smoothed version of MACD used to generate buy/sell signals
        macd_signal_series = macd.macd_signal().iloc[-12:]
        indicators["MACD_Signal"] = dict(
            zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), macd_signal_series)))

        # VWAP helps traders understand average price based on volume
        # Commonly used by institutions to assess fair value
        vwap_series = volume_weighted_average_price(
            data['High'].squeeze(), data['Low'].squeeze(), data['Close'].squeeze(), volume=data['Volume'].squeeze()
        ).iloc[-12:]
        indicators["vwap"] = dict(
            zip(data['Date'].iloc[-12:], map(lambda x: round(x, 2), vwap_series)))

        return {
            'stock_price': data.to_dict(orient='records'),
            'indicators': indicators
        }

    except Exception as e:
        return f"Error fetching price data: {str(e)}"

In [2]:
ticker = "AAPL"  # Example: Apple Inc.

# Fetch historical data (e.g., past 3 months)
stock_data = get_stock_prices(ticker)

C:\Users\44787\AppData\Local\Temp\ipykernel_15100\3750346389.py:4: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  stock_data = get_stock_prices(ticker)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [3]:
stock_data

'No data found for ticker: AAPL'

2. Retrive financial health ratio

```
Metric            | Meaning
pe_ratio          | Price-to-Earnings (valuation)
price_to_book     | Valuation based on book value
debt_to_equity    | Leverage/solvency ratio
profit_margins    | Net income as % of revenue
return_on_equity  | Efficiency of shareholder equity
return_on_assets  | Profitability relative to assets
current_ratio     | Short-term liquidity
quick_ratio       | Liquidity without inventory
gross_margins     | Efficiency at core product level
operating_margins | Profitability from operations
```

In [ ]:
stock = yf.Ticker("AAPL")
info = stock.info
info

In [ ]:
info.get('forwardPE')

In [4]:
@tool
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if not info:
            return f"No financial data found for ticker: {ticker}"

        def safe_get(key: str) -> Union[float, str]:
            value = info.get(key)
            return round(value, 3) if isinstance(value, (int, float)) else "N/A"

        return {
            'pe_ratio': safe_get('forwardPE'),
            'price_to_book': safe_get('priceToBook'),
            'debt_to_equity': safe_get('debtToEquity'),
            'profit_margins': safe_get('profitMargins'),
            'return_on_equity': safe_get('returnOnEquity'),
            'return_on_assets': safe_get('returnOnAssets'),
            'current_ratio': safe_get('currentRatio'),
            'quick_ratio': safe_get('quickRatio'),
            'gross_margins': safe_get('grossMargins'),
            'operating_margins': safe_get('operatingMargins')
        }

    except Exception as e:
        return f"Error fetching ratios: {str(e)}"

In [ ]:
get_financial_metrics("AAPL")

3. get stock news

In [5]:
from typing import Union, List, Dict
from langchain_core.tools import tool
import requests
import os


@tool
def get_stock_news(ticker: str) -> Union[List[Dict], str]:
    """
    Fetches recent news articles related to the given stock ticker using NewsAPI.
    Returns top 5 articles with title, description, source, and URL.
    """
    try:
        url = (
            f"https://newsapi.org/v2/everything?"
            f"q={ticker}&"
            f"sortBy=publishedAt&"
            f"language=en&"
            f"pageSize=5&"
            f"apiKey=00131bbfeba447b7b6d338347ab19c15"
        )

        response = requests.get(url)
        if response.status_code != 200:
            return f"Failed to fetch news: {response.status_code} - {response.text}"

        data = response.json()
        articles = data.get("articles", [])
        if not articles:
            return f"No recent news found for ticker: {ticker}"

        return [
            {
                "title": a["title"],
                "description": a["description"],
                "source": a["source"]["name"],
                "url": a["url"],
                "published_at": a["publishedAt"]
            }
            for a in articles
        ]

    except Exception as e:
        return f"Error fetching news: {str(e)}"


In [ ]:
get_stock_news("AAPL")

``Data Fetched Successfully!``

### AI-generated analysis function using OpenAI's LLM


In [ ]:
'''
from langchain.schema import SystemMessage
from langgraph.graph import START, END
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI

# Define your LLM
llm = ChatOpenAI(model='gpt-4o-mini')

# Define the tools you want to use
tools = [get_stock_prices, get_financial_metrics, get_stock_news]
llm_with_tool = llm.bind_tools(tools)

# Define the graph state structure
class State(TypedDict):
    messages: Annotated[List, add_messages]
    stock: str

# Your refined prompt from previous step
FUNDAMENTAL_ANALYST_PROMPT = """..."""  # (Use the prompt from earlier with `get_stock_news`)

# Define the fundamental analyst step
def fundamental_analyst(state: State):
    messages = [
        SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(
            company=state['stock']
        )),
    ] + state['messages']
    
    return {
        'messages': llm_with_tool.invoke(messages)
    }

# Build the graph
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("fundamental_analyst", fundamental_analyst)
graph_builder.add_node("tools", ToolNode(tools))

# Define how the flow moves
graph_builder.add_edge(START, "fundamental_analyst")
graph_builder.add_conditional_edges("fundamental_analyst", tools_condition)
graph_builder.add_edge("tools", "fundamental_analyst")
graph_builder.add_edge("fundamental_analyst", END)

# Compile the graph
graph = graph_builder.compile()

'''

In [6]:
# import dotenv
# dotenv.load_dotenv()
from langchain_openai import ChatOpenAI

# 1. Init model
llm = ChatOpenAI(model='gpt-4o-mini')

# 2. Bind tools
tools = [get_stock_prices, get_financial_metrics, get_stock_news]
llm_with_tool = llm.bind_tools(tools)

In [7]:
# 3. Prompt Template

FUNDAMENTAL_ANALYST_PROMPT = """
You are a professional fundamental analyst tasked with evaluating a company's (whose symbol is {company}) performance using three types of data:

1. **Stock Prices & Technical Indicators** — provided by `get_stock_prices`
2. **Financial Metrics** — provided by `get_financial_metrics`
3. **Recent News Articles** — provided by `get_stock_news`

You will be given a stock symbol (e.g., AAPL, MSFT) and tool outputs for that stock. Based on these inputs, generate a structured and insightful summary of the stock's current status.

---

### Instructions:
- **Use ONLY the tool-provided data**. Do not fabricate or speculate.
- Focus on trends, patterns, and clear insights.
- Be concise and avoid general financial advice.
- Highlight both strengths and risks.
- Make it useful for someone deciding whether to investigate the stock further.

---

### Your Output Format (JSON-like):
"stock": "<Ticker Symbol>",
"price_analysis": "<Summarize stock price trends and momentum indicators (e.g., RSI, MACD, VWAP)>",
"financial_analysis": "<Summarize financial ratios like P/E, profit margins, debt-to-equity, ROE, etc.>",
"news_analysis": "<Summarize recent headlines, themes, and sentiment from company-related news>",
"final_summary": "<Bring everything together into a clear takeaway or outlook (without recommendations)>",
"asked_question_answer": "<Answer any user question directly using only the information above>"

---

Be factual, data-driven, and structured.
"""


##### creating an instance of StateGraph which will be used to:

- Add nodes (your tools/functions)

- Define edges (the flow between them)

- Set entry and exit points

- Compile the graph into a runnable chain

In [8]:
# setting up a LangGraph using StateGraph with a custom State
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[List, add_messages]
    stock: str

In [9]:
from langchain.schema import SystemMessage

def fundamental_analyst(state: State):
    messages = [
        SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(
            company=state['stock'])),
    ] + state['messages']
    return {
        'messages': llm_with_tool.invoke(messages)
    }

In [10]:
# from langgraph.graph import START, END
# from langgraph.graph import StateGraph
# from langgraph.prebuilt import ToolNode, tools_condition


# graph_builder = StateGraph(State)

# graph_builder.add_node('fundamental_analyst', fundamental_analyst)
# # Add the tool node with a name
# graph_builder.add_node("tools", ToolNode(tools))

# # Connect tool output back to fundamental analysis
# graph_builder.add_edge("tools", "fundamental_analyst")

# # Add the conditional routing based on whether tools are needed
# graph_builder.add_conditional_edges("fundamental_analyst", tools_condition)

# # Ensure start and end points are defined
# graph_builder.add_edge(START, 'fundamental_analyst')
# graph_builder.add_edge("fundamental_analyst", END)  # or loop again if more processing

# # Compile the graph
# graph = graph_builder.compile()

In [11]:
from langgraph.graph import START, END
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition


workflow = StateGraph(State)

workflow.add_node('fundamental_analyst', fundamental_analyst)
workflow.add_edge(START, 'fundamental_analyst')
workflow.add_edge("fundamental_analyst", END)



# Compile the graph
app = workflow.compile()

In [12]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)

HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)


In [ ]:
events = app.stream({'messages': [('user', 'Should I buy this stock?')],
                       'stock': 'AAPL'}, stream_mode='values')
for event in events:
    if 'messages' in event:
        event['messages'][-1].pretty_print()

In [ ]:
import streamlit as st
from langgraph.graph.runner import GraphRunner
from langchain.schema import HumanMessage

# Import your graph and setup from your LangGraph script
from your_graph_file import graph  # Replace with your actual file name
from your_graph_file import State  # State definition

# Streamlit UI
st.set_page_config(page_title="Stock Analysis Assistant", layout="wide")
st.title("📈 AI Stock Analyst")

ticker = st.text_input("Enter stock symbol (e.g., AAPL, TSLA):", value="AAPL")

if st.button("Run Analysis") and ticker:
    st.write("🔍 Analyzing stock data, technical indicators, financial metrics, and news...")

    # Initialize the graph
    runner = GraphRunner(graph)

    # Define initial state
    initial_state: State = {
        "stock": ticker.upper(),
        "messages": [HumanMessage(content=f"Can you analyze {ticker.upper()} for me?")]
    }

    # Run the graph
    final_state = runner.invoke(initial_state)

    # Extract message
    last_message = final_state["messages"][-1].content

    # Display result
    st.markdown("### 📊 Fundamental Analysis Summary")
    st.markdown(last_message)


In [ ]:
# 📁 app.py
import streamlit as st
from langchain.schema import HumanMessage
from your_langgraph_module import graph  # <- Import your graph from your existing LangGraph setup

# Streamlit page config
st.set_page_config(page_title="Stock Analyzer", layout="centered")

st.title("📈 AI Stock Analyzer")

# User input section
stock = st.text_input("Enter Stock Symbol (e.g., AAPL):", value="AAPL")
user_question = st.chat_input("Ask your stock analysis question:")

# Show previous chat history if needed
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# On user input
if user_question and stock:
    # Show user message
    with st.chat_message("user"):
        st.write(user_question)

    # Append to chat history
    st.session_state.chat_history.append(HumanMessage(content=user_question))

    # Trigger the graph stream
    with st.chat_message("assistant"):
        message_container = st.empty()  # This will update as we stream

        messages = []
        for event in graph.stream({
            "messages": st.session_state.chat_history,
            "stock": stock
        }, stream_mode="values"):
            if 'messages' in event:
                latest = event['messages'][-1]
                content = latest.content if hasattr(latest, 'content') else str(latest)
                messages.append(content)
                # Stream output live
                message_container.markdown("\n\n".join(messages))

        # Add assistant's last message to session history (optional)
        st.session_state.chat_history.append(event['messages'][-1])
